# Setup

In [1]:
%load_ext edx_jupyter

In [2]:
%%vertica
SELECT 'Connected to Vertica!'

,?column?
0,Connected to Vertica!


In [3]:
import datetime
from dateutil.tz import tzutc

# Configuration

## Enrollment Date Config

In [4]:
# enter enrollment start date
enrollment_start = datetime.datetime(2017, 7, 4, 0, 0, tzinfo=tzutc())

## Segmentation Config

In [5]:
# enter number of user segments (i.e., total number of treatment groups + 1 for the control group)
num_user_segments = 2

# Segmented Enrollments

In [6]:
%%vertica
SET TIME ZONE TO UTC

,?column?


In [7]:
%%vertica (enrollment_start=enrollment_start, num_user_segments=num_user_segments)

--DROP TABLE IF EXISTS ret.exp6_enrollments;
CREATE TABLE IF NOT EXISTS ret.exp6_enrollments
(
    segment_id INT,
    user_id INT,
    course_id CHARACTER VARYING,
    first_enrollment_time TIMESTAMP,
    content_availability TIMESTAMP
);
--GRANT ALL PRIVILEGES ON ret.exp6_enrollments TO rapid_experiments_team;

,?column?


In [8]:
%%vertica (enrollment_start=enrollment_start, num_user_segments=num_user_segments)

INSERT INTO ret.exp6_enrollments
SELECT
    ROW_NUMBER() OVER (PARTITION BY 1 ORDER BY first_enrollment_time) % :num_user_segments as segment_id,    
    uc.user_id,
    uc.course_id,
    uc.first_enrollment_time,
    uc.content_availability
FROM
(
    SELECT
        user_id,
        course_id,
        first_enrollment_time,
        content_availability
    FROM
    (
        SELECT
            user_id,
            uc.course_id,
            first_enrollment_time,
            GREATEST(c.course_start_date::TIMESTAMPTZ, uc.first_enrollment_time::TIMESTAMPTZ) AS content_availability,
            ROW_NUMBER() over (PARTITION BY user_id ORDER BY first_enrollment_time DESC) AS row
        FROM production.d_user_course uc
        JOIN
            business_intelligence.course_master c
        ON uc.course_id = c.course_id
        AND c.pacing_type = 'self_paced'
        AND (c.content_language IS NULL OR c.content_language LIKE 'en%')
        AND c.is_WL = 0
        AND GREATEST(c.course_start_date::TIMESTAMPTZ, uc.first_enrollment_time::TIMESTAMPTZ) BETWEEN :enrollment_start AND NOW()
    ) AS uc
    WHERE row = 1
) AS uc
LEFT JOIN
(
    SELECT
        user_id,
        course_id
    FROM ret.exp6_enrollments
) AS already_inserted
ON uc.user_id = already_inserted.user_id
AND uc.course_id = already_inserted.course_id
WHERE already_inserted.user_id IS NULL
ORDER BY first_enrollment_time

,OUTPUT
0,69630


In [9]:
%vertica select COUNT(*), COUNT(DISTINCT user_id) from ret.exp6_enrollments

,COUNT,COUNT
0,69630,69630


In [10]:
%vertica COMMIT

,COUNT,COUNT


In [11]:
%vertica SELECT * FROM ret.exp6_enrollments LIMIT 10

,segment_id,user_id,course_id,first_enrollment_time,content_availability
0,1,11255793,course-v1:PennX+ROBO3x+2T2017,2017-01-08 23:04:01.013817,2017-07-10
1,0,12755581,course-v1:UWashingtonX+CYB002x+1T2017,2017-01-17 21:56:05.309849,2017-07-10
2,1,4230805,course-v1:UWashingtonX+CYB002x+1T2017,2017-01-17 21:56:51.301240,2017-07-10
3,0,13196290,course-v1:UWashingtonX+CYB002x+1T2017,2017-01-25 00:38:49.017025,2017-07-10
4,1,2669637,course-v1:UWashingtonX+CYB002x+1T2017,2017-01-25 00:40:22.910184,2017-07-10
5,0,12696834,course-v1:UWashingtonX+CYB002x+1T2017,2017-01-25 00:41:43.269306,2017-07-10
6,1,977228,course-v1:UWashingtonX+CYB002x+1T2017,2017-01-25 00:43:41.084022,2017-07-10
7,0,12437350,course-v1:PennX+ROBO2x+2T2017,2017-02-14 13:26:15.752425,2017-07-10
8,1,3812979,course-v1:PennX+ROBO2x+2T2017,2017-02-27 22:41:36.452885,2017-07-10
9,0,61097,course-v1:PennX+ROBO3x+2T2017,2017-02-27 23:16:27.063188,2017-07-10


In [12]:
%vertica SELECT segment_id, COUNT(*) FROM ret.exp6_enrollments GROUP BY segment_id

,segment_id,COUNT
0,0,34815
1,1,34815
